In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import PyPDFDirectoryLoader

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS

from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain.prompts import PromptTemplate


from langchain.chains import RetrievalQA

In [8]:
loader = PyPDFDirectoryLoader("./reliability")

documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
final_documents = text_splitter.split_documents(documents)
len(final_documents)


11

In [10]:
Huggingface_embeddings = HuggingFaceBgeEmbeddings(
    model_name="BAAI/bge-small-en-v1.5", 
    model_kwargs={"device": "cpu"},
    encode_kwargs={"normalize_embeddings": True}
)

In [15]:
import  numpy as np
print(np.array(Huggingface_embeddings.embed_query(final_documents[0].page_content)))
print(np.array(Huggingface_embeddings.embed_query(final_documents[0].page_content)).shape)

[-4.45106328e-02 -2.08494905e-02  1.64920650e-03 -1.06336800e-02
  3.61050479e-02  2.28325855e-02  1.16156891e-01  2.23064553e-02
  3.57013084e-02  1.03948440e-03  5.20878136e-02 -4.45944406e-02
  7.52096176e-02 -1.51128098e-02 -1.36651052e-02  5.11960611e-02
 -5.10529689e-02  4.88237515e-02  6.77977651e-02 -6.25571758e-02
  1.05117142e-01 -2.29992904e-02 -1.09677268e-02 -1.78853739e-02
  3.67015041e-02  6.49010912e-02 -3.66079248e-02  2.02322677e-02
 -1.02022775e-02 -1.25149742e-01 -3.49860527e-02  6.06301706e-03
 -6.28810450e-02 -7.15822428e-02 -1.33577716e-02 -2.73708608e-02
  9.07621067e-03 -2.34484598e-02 -5.45840652e-04  5.71681699e-03
 -5.38177267e-02  8.66935682e-03 -1.90860834e-02 -5.22610359e-02
 -3.35520543e-02 -1.43058859e-02 -4.52715624e-03 -4.21434939e-02
 -9.98236518e-03  3.33234593e-02  2.27042176e-02  1.06239934e-02
  6.61173612e-02  5.33741005e-02  6.42898399e-03  3.21992598e-02
  8.95208046e-02  7.33826682e-02  3.03784572e-02  1.42466975e-04
  4.36370298e-02 -2.62018

In [17]:
vectorstore=FAISS.from_documents(final_documents[:120],Huggingface_embeddings)

In [18]:
query="What is reliability?"
relevant_docments=vectorstore.similarity_search(query)

print(relevant_docments[0].page_content)

MAT3003-Probability, Statictics and Reliability CW
Reliability Engineering Concepts
1. Reliability R(t)
Reliability is a measure of the ability of a system or component to perform its required functions under
stated conditions for a specified period of time. Mathematically, reliability is defined as the probability that
a system or component does not fail within a given time t.
The reliability function is expressed as:
R(t) = P(T > t)
where T is the random variable representing the time to failure.
2. Cumulative Failure Function F(t)
The cumulative failure function, also known as the cumulative distribution function (CDF) of the time to
failure, represents the probability that the system or component will fail by a certain time t. It is the
complement of the reliability function.
The cumulative failure function is expressed as:
F(t) = P(T ≤ t) = 1 − R(t)
3. Failure Density Function f(t)


In [19]:
retriever=vectorstore.as_retriever(search_type="similarity",search_kwargs={"k":3})
print(retriever)

tags=['FAISS', 'HuggingFaceBgeEmbeddings'] vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001A0ED636A50> search_kwargs={'k': 3}


In [22]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Get the token with error checking
hf_token = os.getenv("HUGGINGFACEHUB_API_TOKEN")

In [39]:
#Hugging Face models can be run locally through the HuggingFacePipeline class.
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline

# Use an unrestricted model
hf = HuggingFacePipeline.from_model_id(
    model_id="microsoft/DialoGPT-medium",  # Open access model
    task="text-generation",
    pipeline_kwargs={"temperature": 0, "max_new_tokens": 300}
)

llm = hf 
response = llm.invoke(query)
print(response)

tokenizer_config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

c:\Users\ppriy\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ppriy\.cache\huggingface\hub\models--microsoft--DialoGPT-medium. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Device set to use cpu


ValueError: `temperature` (=0) has to be a strictly positive float, otherwise your next token scores will be invalid.

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/863M [00:00<?, ?B/s]